In [1]:
import veroviz as vrv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from ipyleaflet import Map, Marker, Polyline
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import json
import requests
import os
import polyline
import geopy.distance

In [2]:
# Setting up matplotlib to display inline in Jupyter notebooks
%matplotlib inline

In [3]:
def get_osrm_route(start, end, server='http://router.project-osrm.org'):
    coords = f"{start[1]},{start[0]};{end[1]},{end[0]}"
    url = f"{server}/route/v1/driving/{coords}?overview=full"
    try:
        response = requests.get(url).json()
        if response['code'] == 'Ok':
            encoded_polyline = response['routes'][0]['geometry']
            route = polyline.decode(encoded_polyline)
            return [{'coordinates': {'lat': pt[0], 'lng': pt[1]}} for pt in route], None
        else:
            # Return a more descriptive error message
            return None, f"Error from OSRM: {response.get('message', 'No specific error message')}"
    except Exception as e:
        # Return the exception message itself if an exception occurs
        return None, f"Exception in getting OSRM route: {str(e)}"

In [4]:
def load_json_files(directory):
    # Map of route IDs to city names
    route_to_city = {
        "0ba52062-f98a-4750-a6ef-0a72ff35202e": "Chicago",
        "69233ee9-1ac4-4d89-b322-94fb13df54a3": "LA",
        "02f91cfa-3839-4e55-91a0-ab19c3e77683": "Seattle",
        "ed9dd222-9ce7-47f3-8cd9-d2a5d2b6a493": "Austin",
        "e9939a49-8f0d-439f-80d9-1840dc1376ca": "Boston"
    }

    data_objects = {}
    for filename in os.listdir(directory):
        if filename.endswith('.json'):  # Ensure it's a JSON file
            # Extract the route ID from the filename
            route_id = filename.split('_')[1]
            if route_id in route_to_city:
                file_path = os.path.join(directory, filename)
                with open(file_path, 'r') as file:
                    file_data = json.load(file)
                    city_name = route_to_city[route_id]
                    # Combine all events under each city name
                    if city_name not in data_objects:
                        data_objects[city_name] = []
                    data_objects[city_name].extend(file_data)  # Append events directly

    return data_objects

# Usage example with paths to directories
vehicle_only_events_directory_path = '../algorithm/amazon_vehicle_events'
vehicle_only_paths_directory_path = '../algorithm/amazon_vehicle_paths'

vehicle_only_events = load_json_files(vehicle_only_events_directory_path)
vehicle_only_paths = load_json_files(vehicle_only_paths_directory_path)


In [5]:
for key, value in vehicle_only_paths.items():
    print({key:value})

{'Boston': [{'id': 1, 'stop': 'SW', 'coordinates': {'lat': 42.139891, 'lng': -71.494346}}, {'id': 2, 'stop': 'DO', 'coordinates': {'lat': 42.207436, 'lng': -71.684275}}, {'id': 3, 'stop': 'SQ', 'coordinates': {'lat': 42.208183, 'lng': -71.683617}}, {'id': 4, 'stop': 'IA', 'coordinates': {'lat': 42.209057, 'lng': -71.682809}}, {'id': 5, 'stop': 'KG', 'coordinates': {'lat': 42.209494, 'lng': -71.682438}}, {'id': 6, 'stop': 'IB', 'coordinates': {'lat': 42.210583, 'lng': -71.682959}}, {'id': 7, 'stop': 'SS', 'coordinates': {'lat': 42.210652, 'lng': -71.681458}}, {'id': 8, 'stop': 'ZH', 'coordinates': {'lat': 42.211819, 'lng': -71.680066}}, {'id': 9, 'stop': 'XW', 'coordinates': {'lat': 42.212, 'lng': -71.677426}}, {'id': 10, 'stop': 'GC', 'coordinates': {'lat': 42.212232, 'lng': -71.675846}}, {'id': 11, 'stop': 'SF', 'coordinates': {'lat': 42.218855, 'lng': -71.677317}}, {'id': 12, 'stop': 'FR', 'coordinates': {'lat': 42.206324, 'lng': -71.677589}}, {'id': 13, 'stop': 'WR', 'coordinates': 

In [6]:
vehicle_only_events['Chicago']

[{'id': 1,
  'event_type': 'DEPART',
  'stop_id': 'VE',
  'time': 0,
  'coordinates': {'lat': 42.031368, 'lng': -87.776596}},
 {'id': 2,
  'event_type': 'ARRIVE',
  'stop_id': 'TG',
  'time': 644.2,
  'coordinates': {'lat': 42.040283, 'lng': -87.732541}},
 {'id': 3,
  'event_type': 'SERVICE_START',
  'stop_id': 'TG',
  'time': 644.2,
  'coordinates': {'lat': 42.040283, 'lng': -87.732541}},
 {'id': 4,
  'event_type': 'SERVICE_END',
  'stop_id': 'TG',
  'time': 737.0,
  'coordinates': {'lat': 42.040283, 'lng': -87.732541}},
 {'id': 5,
  'event_type': 'DEPART',
  'stop_id': 'TG',
  'time': 737.0,
  'coordinates': {'lat': 42.040283, 'lng': -87.732541}},
 {'id': 6,
  'event_type': 'ARRIVE',
  'stop_id': 'GP',
  'time': 784.7,
  'coordinates': {'lat': 42.040451, 'lng': -87.73142}},
 {'id': 7,
  'event_type': 'SERVICE_START',
  'stop_id': 'GP',
  'time': 784.7,
  'coordinates': {'lat': 42.040451, 'lng': -87.73142}},
 {'id': 8,
  'event_type': 'SERVICE_END',
  'stop_id': 'GP',
  'time': 831.7,

In [7]:
def process_single_city_events(events):
    """Process the event data for a single city and return a DataFrame with structured data."""
    # Normalize data and create DataFrame for each event list independently
    df = pd.DataFrame(events)

    # Calculate and handle each attribute, ensuring all entries are processed independently
    processed_df = pd.DataFrame({
        'id': df['id'],
        'lat': df['coordinates'].apply(lambda x: x['lat']),
        'lon': df['coordinates'].apply(lambda x: x['lng']),
        'elevMeters': 0,
        'objectID': df['id'],
        'startTimeSec': df['time'],
        'nodeName': df.apply(lambda x: f"Node {x['id']}", axis=1),
        'nodeType': 'Waypoint',
        'popupText': df.apply(lambda x: f"{x['id']} Event: {x['event_type']}<br>Time: {x['time']}", axis=1),
        'leafletIconPrefix': 'fa',
        'leafletIconType': df['id'].apply(lambda x: 'flag' if x % 2 == 0 else 'circle'),
        'leafletColor': df['id'].apply(lambda x: 'green' if x % 2 == 0 else 'blue'),
        'leafletIconText': df['id'].apply(str),
        'cesiumIconType': 'pin',
        'cesiumColor': df['id'].apply(lambda x: 'red' if x % 2 == 0 else 'yellow'),
        'cesiumIconText': df['id'].apply(str),
        'altMeters': 0,
        'stopID': df['stop_id']
    })

    return processed_df

# Dictionary to hold processed DataFrames for each city
vehicle_only_events_dataframes = {}

# Loop over each city and its associated events data
for city, events in vehicle_only_events.items():
    vehicle_only_events_dataframes[city] = process_single_city_events(events)


In [8]:
vehicle_only_nodes = vehicle_only_events_dataframes

In [9]:
mp = {}
for city, df in vehicle_only_nodes.items():
    mp[city] = {}
    for i, row in df.iterrows():
        mp[city][row['stopID']] = i

In [10]:
vehicle_props = {
    'Austin': {'modelFile': 'truck.glb', 'color': 'red'},
    'Chicago': {'modelFile': 'truck.glb', 'color': 'blue'},
    'Seattle': {'modelFile': 'truck.glb', 'color': 'green'},
    'Boston': {'modelFile': 'truck.glb', 'color': 'yellow'},
    'LA': {'modelFile': 'truck.glb', 'color': 'purple'}
}

def create_vehicle_only_rows(odID, objectID, vehicle_props, route, times):
    if not route:
        print(f"Empty route received for {objectID}.")
        return []

    rows = []
    for i in range(len(route) - 1):
        start_point = route[i]['coordinates']
        end_point = route[i+1]['coordinates']
        row = {
            'odID': odID + i,
            'objectID': objectID,
            'modelFile': vehicle_props[objectID]['modelFile'],
            'startLat': start_point['lat'],
            'startLon': start_point['lng'],
            'endLat': end_point['lat'],
            'endLon': end_point['lng'],
            'startElevMeters': 0,  # Example default value
            'endElevMeters': 0,
            'startAltMeters': 100 if objectID == 'drone' else 0,  # Example customization for drones
            'endAltMeters': 100 if objectID == 'drone' else 0,
            'startTimeSec': times[i],
            'endTimeSec': times[i+1],
            'leafletColor': vehicle_props[objectID]['color'],
            'leafletWeight': 5,
            'leafletStyle': 'solid',
            'leafletOpacity': 0.8,
            'leafletCurveType': 'straight',
            'leafletCurvature': 0,
            'arcCurveType': 'straight',
            'useArrows': True,
            'cesiumColor': vehicle_props[objectID]['color'],
            'cesiumWeight': 5,
            'cesiumStyle': 'solid',
            'cesiumOpacity': 0.8,
            'popupText': f'Route segment {i+1} for {objectID}'
        }
        rows.append(row)

    return rows


def process_vehicle_paths(vehicle_paths, vehicle_props):
    all_rows = []
    odID = 1

    for objectID, path in vehicle_paths.items():
        print(f"Processing path for {objectID} with {len(path)} stops.")
        for i in range(len(path) - 1):
            start = path[i]['coordinates']
            end = path[i+1]['coordinates']
            route, error = get_osrm_route((start['lat'], start['lng']), (end['lat'], end['lng']))
            if error:
                print(f"Error processing segment {i+1} in {objectID}: {error}")
                continue  # Skip this segment due to error

            # interpolate times
            times = []
            start_time = vehicle_only_nodes[objectID].loc[mp[objectID][path[i]['stop']], 'startTimeSec']
            end_time = vehicle_only_nodes[objectID].loc[mp[objectID][path[i+1]['stop']], 'startTimeSec']
            if route:
                tot = 0
                for i in range(len(route) - 1):
                    di = geopy.distance.distance((start['lat'], start['lng']), (end['lat'], end['lng'])).km
                    tot += di
                times.append(start_time)
                for i in range(len(route) - 1):
                    di = geopy.distance.distance((start['lat'], start['lng']), (end['lat'], end['lng'])).km
                    times.append(times[-1] if tot == 0 else times[-1] + (end_time - start_time) * di / tot)

            rows = create_vehicle_only_rows(odID, objectID, vehicle_props, route, times)
            all_rows.extend(rows)
            odID += len(rows)

    return all_rows


In [11]:
def validate_route_data(routes):
    for i, route in enumerate(routes):
        if 'stop' not in route or 'coordinates' not in route or 'id' not in route:
            print(f"Data missing in route at index {i}: {route}")
        if 'lat' not in route['coordinates'] or 'lng' not in route['coordinates']:
            print(f"Coordinate data missing in route at index {i}: {route}")

# Example use:
validate_route_data(vehicle_only_paths['Boston'])

In [12]:
vehicle_only_rows = process_vehicle_paths(vehicle_only_paths, vehicle_props)

Processing path for Boston with 167 stops.
Processing path for Chicago with 204 stops.


In [ ]:
vehicle_only_assignments = vehicle_only_rows

In [ ]:
for key, value in vehicle_only_paths.items():
    print(len(value))

204
139


In [ ]:
len(vehicle_only_rows)

3769

In [ ]:
vehicle_only_assignments

[{'odID': 1,
  'objectID': 'Chicago',
  'modelFile': 'truck.glb',
  'startLat': 42.03137,
  'startLon': -87.77661,
  'endLat': 42.03186,
  'endLon': -87.7766,
  'startElevMeters': 0,
  'endElevMeters': 0,
  'startAltMeters': 0,
  'endAltMeters': 0,
  'startTimeSec': 23888.999999999996,
  'endTimeSec': 23696.066666666662,
  'leafletColor': 'blue',
  'leafletWeight': 5,
  'leafletStyle': 'solid',
  'leafletOpacity': 0.8,
  'leafletCurveType': 'straight',
  'leafletCurvature': 0,
  'arcCurveType': 'straight',
  'useArrows': True,
  'cesiumColor': 'blue',
  'cesiumWeight': 5,
  'cesiumStyle': 'solid',
  'cesiumOpacity': 0.8,
  'popupText': 'Route segment 1 for Chicago'},
 {'odID': 2,
  'objectID': 'Chicago',
  'modelFile': 'truck.glb',
  'startLat': 42.03186,
  'startLon': -87.7766,
  'endLat': 42.03239,
  'endLon': -87.77658,
  'startElevMeters': 0,
  'endElevMeters': 0,
  'startAltMeters': 0,
  'endAltMeters': 0,
  'startTimeSec': 23696.066666666662,
  'endTimeSec': 23503.133333333328,
 

In [ ]:
vehicle_naive_events_directory_path = '../algorithm/naive_vehicle_events'
vehicle_naive_paths_directory_path = '../algorithm/naive_vehicle_paths'
drone_naive_events_directory_path = '../algorithm/naive_drone_events'
drone_naive_paths_directory_path = '../algorithm/naive_drone_paths'

def load_json_files(directory):
    data_objects = {}
    for filename in os.listdir(directory):
        if filename.endswith('.json'):  # Ensure it's a JSON file
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r') as file:
                data_objects[filename] = json.load(file)
    return data_objects

vehicle_naive_events = load_json_files(vehicle_naive_events_directory_path)
vehicle_naive_paths = load_json_files(vehicle_naive_paths_directory_path)
drone_naive_events = load_json_files(drone_naive_events_directory_path)
drone_naive_paths = load_json_files(drone_naive_paths_directory_path)

In [ ]:
vehicle_optimize_events_directory_path = '../algorithm/optimized_vehicle_events'
vehicle_optimize_paths_directory_path = '../algorithm/optimized_vehicle_paths'
drone_optimize_events_directory_path = '../algorithm/optimized_drone_events'
drone_optimize_paths_directory_path = '../algorithm/optimized_drone_paths'

def load_json_files(directory):
    data_objects = {}
    for filename in os.listdir(directory):
        if filename.endswith('.json'):  # Ensure it's a JSON file
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r') as file:
                data_objects[filename] = json.load(file)
    return data_objects

vehicle_optimize_events = load_json_files(vehicle_optimize_events_directory_path)
vehicle_optimize_paths = load_json_files(vehicle_optimize_paths_directory_path)
drone_optimize_events = load_json_files(drone_optimize_events_directory_path)
drone_optimize_paths = load_json_files(drone_optimize_paths_directory_path)

In [ ]:
 #Initialize a new DataFrame for storing nodes using VeRoViz's initDataframe function
newNodes = vrv.initDataframe('Nodes')
# Initialize a new DataFrame for storing arcs using VeRoViz's initDataframe function
newArcs = vrv.initDataframe('Arcs')
#newArcs
newNodes

,id,lat,lon,altMeters,nodeName,nodeType,popupText,leafletIconPrefix,leafletIconType,leafletColor,leafletIconText,cesiumIconType,cesiumColor,cesiumIconText,elevMeters


In [ ]:
# Dictionary to store 3D model files and color properties for different vehicle types
vehicleProperties = {
    'drone': {
        'modelFile': 'veroviz/models/drone.gltf',  # Path to the 3D model file for drones
        'color': 'red'                             # Display color for drone models in visualizations
    },
    'truck': {
        'modelFile': 'veroviz/models/ub_truck.gltf',  # Path to the 3D model file for trucks
        'color': 'blue'                               # Display color for truck models in visualizations
    }
}

In [ ]:
nodes_df = vehicle_only_nodes

In [ ]:


icons = {
    'drone': 'drone.png',  # Path to your drone icon
    'truck': 'delivery-truck.png'   # Path to your truck icon
}

# Apply icons and popup text
nodes_df['iconURL'] = nodes_df['objectID'].apply(lambda x: icons[x])
nodes_df['popupText'] = nodes_df.apply(
    lambda row: f"<b>Segment Start</b><br>Time: {row['startTimeSec']}<br>Object ID: {row['objectID']}",
    axis=1
)

combined_map_object = vrv.createLeaflet(
    nodes=nodes_df,
    arcs=route_assignments
)

combined_map_object

In [ ]:
vehicle_only_assignments = pd.DataFrame(vehicle_only_assignments)

In [ ]:
vehicle_only_assignments['modelFile'] = 'veroviz/models/ub_truck.gltf'

In [ ]:
vehicle_only_assignments

,odID,objectID,modelFile,startLat,startLon,endLat,endLon,startElevMeters,endElevMeters,startAltMeters,...,leafletOpacity,leafletCurveType,leafletCurvature,arcCurveType,useArrows,cesiumColor,cesiumWeight,cesiumStyle,cesiumOpacity,popupText
0,1,Chicago,veroviz/models/ub_truck.gltf,42.03137,-87.77661,42.03186,-87.77660,0,0,0,...,0.8,straight,0,straight,True,blue,5,solid,0.8,Route segment 1 for Chicago
1,2,Chicago,veroviz/models/ub_truck.gltf,42.03186,-87.77660,42.03239,-87.77658,0,0,0,...,0.8,straight,0,straight,True,blue,5,solid,0.8,Route segment 2 for Chicago
2,3,Chicago,veroviz/models/ub_truck.gltf,42.03239,-87.77658,42.03277,-87.77657,0,0,0,...,0.8,straight,0,straight,True,blue,5,solid,0.8,Route segment 3 for Chicago
3,4,Chicago,veroviz/models/ub_truck.gltf,42.03277,-87.77657,42.03356,-87.77655,0,0,0,...,0.8,straight,0,straight,True,blue,5,solid,0.8,Route segment 4 for Chicago
4,5,Chicago,veroviz/models/ub_truck.gltf,42.03356,-87.77655,42.03367,-87.77655,0,0,0,...,0.8,straight,0,straight,True,blue,5,solid,0.8,Route segment 5 for Chicago
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3764,3765,LA,veroviz/models/ub_truck.gltf,33.91710,-118.32645,33.91739,-118.32644,0,0,0,...,0.8,straight,0,straight,True,purple,5,solid,0.8,Route segment 694 for LA
3765,3766,LA,veroviz/models/ub_truck.gltf,33.91739,-118.32644,33.91760,-118.32644,0,0,0,...,0.8,straight,0,straight,True,purple,5,solid,0.8,Route segment 695 for LA
3766,3767,LA,veroviz/models/ub_truck.gltf,33.91760,-118.32644,33.91773,-118.32644,0,0,0,...,0.8,straight,0,straight,True,purple,5,solid,0.8,Route segment 696 for LA
3767,3768,LA,veroviz/models/ub_truck.gltf,33.91773,-118.32644,33.91801,-118.32644,0,0,0,...,0.8,straight,0,straight,True,purple,5,solid,0.8,Route segment 697 for LA


In [ ]:
vehicle_only_assignments['startTimeSec']

0       23889.000000
1       23696.066667
2       23503.133333
3       23310.200000
4       23117.266667
            ...     
3764    26691.237106
3765    26695.089685
3766    26698.942264
3767    26702.794842
3768    26706.647421
Name: startTimeSec, Length: 3769, dtype: float64

In [ ]:
vehicle_only_assignments['endTimeSec']

0       23696.066667
1       23503.133333
2       23310.200000
3       23117.266667
4       22924.333333
            ...     
3764    26695.089685
3765    26698.942264
3766    26702.794842
3767    26706.647421
3768    26710.500000
Name: endTimeSec, Length: 3769, dtype: float64

In [ ]:
assignments = vehicle_only_assignments
assignments['modelScale'] = 100
assignments['modelMinPxSize'] = 75
assignments['ganttColor'] = 'darkgray'
assignments['wayname'] = None
assignments['waycategory'] = None
assignments['surface'] = None
assignments['waytype'] = None
assignments['steepness'] = None
assignments['tollway'] = None

In [ ]:
vrv.createCesium(assignments=assignments,
                 nodes=nodes_df,
                 cesiumDir=os.getcwd()+'/Cesium-1.116',
                 problemDir='veroviz/new')

/home/max/miniconda3/envs/vrpd/lib/python3.12/site-packages/veroviz/createCesium.py:768: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  path = path.append({
/home/max/miniconda3/envs/vrpd/lib/python3.12/site-packages/veroviz/createCesium.py:768: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  path = path.append({
/home/max/miniconda3/envs/vrpd/lib/python3.12/site-packages/veroviz/createCesium.py:768: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  path = path.append({
/home/max/miniconda3/envs/vrpd/lib/python3.12/site-packages/veroviz/createCesium.py:768: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  path = path.append({
/home/max/miniconda3

Message: File selector was written to /home/max/vrpd-amazon-dataset/visual/Cesium-1.116/veroviz/new/;veroviz;new.vrv ...
Message: Configs were written to /home/max/vrpd-amazon-dataset/visual/Cesium-1.116/veroviz/new/config.js ...
Message: Nodes were written to /home/max/vrpd-amazon-dataset/visual/Cesium-1.116/veroviz/new/displayNodes.js ...
Message: Assignments (.js) were written to /home/max/vrpd-amazon-dataset/visual/Cesium-1.116/veroviz/new/displayPaths.js ...
Message: Assignments (.czml) were written to /home/max/vrpd-amazon-dataset/visual/Cesium-1.116/veroviz/new/routes.czml ...
